# Salary Prediction Project: What leads to a higher salary?

In [1]:
#personal info
__author__ = "Cici Du"
__email__ = "cicidhz@gmail.com"

#### Import libraries

In [2]:
%matplotlib inline

import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from zipfile import ZipFile, Path

#### Load datasets

In [3]:
with ZipFile("../data/data.zip") as zip:
    train_salary = zip.open('data/train_salaries.csv')
    train_features = zip.open('data/train_features.csv')
    test_features = zip.open('data/test_features.csv')
train_salary = pd.read_csv(train_salary)
train_features = pd.read_csv(train_features)
test_features = pd.read_csv(test_features)

In [4]:
#merge datasets
train_df = pd.merge(train_salary, train_features, how='inner', on='jobId')

#### Clean the data

In [5]:
#clean data
def clean_data(df):
    '''remove jobId and companyId columns, remove records with salary <= 0'''
    if 'jobId' in df.columns:
        df.drop(['jobId'], axis = 1, inplace = True)
    else:
        pass
    df.drop(['companyId'], axis = 1, inplace = True)
    new_df = df.loc[train_df['salary'] > 0]
    return new_df

In [6]:
#assign categorical and numeric features
cat_var = ['jobType','degree','major','industry']
num_var = ['yearsExperience','milesFromMetropolis']
tar_var = 'salary'

def dummy_encode(df,cat_var,num_var):
    cat_df = pd.get_dummies(df[cat_var], drop_first=True)
    num_df = df[num_var]
    return pd.concat([num_df,cat_df,], axis=1)

In [7]:
clean_df = clean_data(train_df)
encoded_df = dummy_encode(clean_df,cat_var, num_var)

In [8]:
x_train = encoded_df
y_train = clean_df[tar_var]

### Train models 

In [9]:
#Use MSE as evaluation metric
def model_average_mse(model):
    nmse = cross_val_score(model, x_train,y_train, scoring = 'neg_mean_squared_error')
    mean_mse = -1*np.mean(nmse)
    print(model)
    print('Average MSE is ' + str(mean_mse))

In [10]:
#create and train a linear regression model
lr = LinearRegression()
lr_model = lr.fit(x_train,y_train)
model_average_mse(lr_model)

LinearRegression()
Average MSE is 384.4143373223809


In [11]:
#create and train a random forest regressor model
#to avoid memory issues during tuning, set the n_estimators as 50
rfg_tune = RandomForestRegressor(n_estimators = 50, oob_score = True)
#further split the train dataset to tune the random forest regressor
x_model_train, x_model_test, y_model_train, y_model_test = train_test_split(x_train,y_train, random_state = 1)

In [12]:
rf_model = rfg_tune.fit(x_model_train, y_model_train)
rf_model.oob_score_

0.6936717321642283

In [13]:
max_depth_list = [10,15,20,25,30]

for depth in max_depth_list:
    rfg_tune = RandomForestRegressor(n_estimators=50, oob_score=True, random_state=1, n_jobs=2, max_depth=depth)
    rf_model = rfg_tune.fit(x_model_train, y_model_train)
    print('depth is '+str(depth)+' OOB Score is '+str(rf_model.oob_score_))

depth is 10 OOB Score is 0.6827495167169831
depth is 15 OOB Score is 0.7392001518853081
depth is 20 OOB Score is 0.7353253480428954
depth is 25 OOB Score is 0.7075496861995503
depth is 30 OOB Score is 0.6946296693137604


In [14]:
#Tune max features
max_features_list = ['auto','sqrt','log2', 0.7, 0.8, 0.9]

for max_features in max_features_list:
    rfg_tune = RandomForestRegressor(n_estimators=50, oob_score=True, random_state=1, n_jobs=2, max_depth=15, max_features = max_features)
    rf_model = rfg_tune.fit(x_model_train, y_model_train)
    print('max_features is '+str(max_features)+' OOB Score is '+str(rf_model.oob_score_))

max_features is auto OOB Score is 0.7392001518853081
max_features is sqrt OOB Score is 0.7284357345238022
max_features is log2 OOB Score is 0.7190039230327228
max_features is 0.7 OOB Score is 0.742479279118901
max_features is 0.8 OOB Score is 0.7418998215358138
max_features is 0.9 OOB Score is 0.7403555732798361


In [15]:
#Tune min_samples_leaf
min_samples_leaf_list = [1, 5, 10, 20, 30, 50]

for min_samples_leaf in min_samples_leaf_list:
    rfg_tune = RandomForestRegressor(n_estimators=50, oob_score=True, random_state=1, n_jobs=2, max_depth=15, max_features = 0.7, min_samples_leaf = min_samples_leaf)
    rf_model = rfg_tune.fit(x_model_train, y_model_train)
    print('min_samples_leaf is '+str(min_samples_leaf)+' OOB Score is '+str(rf_model.oob_score_))

min_samples_leaf is 1 OOB Score is 0.742479279118901
min_samples_leaf is 5 OOB Score is 0.743432220042391
min_samples_leaf is 10 OOB Score is 0.7437120224839666
min_samples_leaf is 20 OOB Score is 0.7435153501623668
min_samples_leaf is 30 OOB Score is 0.7432229874978737
min_samples_leaf is 50 OOB Score is 0.7418509787997235


In [ ]:
#finalize the random forest regressor model
#increase the number of trees for the final model
rfg = RandomForestRegressor(n_estimators=150, oob_score=True, random_state=1, n_jobs=2, max_depth=15, max_features = 0.7, min_samples_leaf = 10)
rf_model = rfg.fit(x_train,y_train)
model_average_mse(rf_model)

In [ ]:
#Visualize feature importances
feature_importances = pd.Series(rf_model.feature_importances_, index = x_train.columns)
feature_importances.sort_values(inplace = True)
feature_importances.plot(kind = 'barh', figsize=(10,8))

In [ ]:
def combine_feature_importances(model, features, sum_cols):
  feature_dict = dict(zip(features, model.feature_importances_))

  for col in sum_cols:
    sum_val = sum(x for i, x in feature_dict.items() if col in i)
    remove = [i for i in feature_dict.keys() if col in i]
    for i in remove:
      feature_dict.pop(i)
    feature_dict[col] = sum_val 
  results = pd.Series(feature_dict, index = feature_dict.keys())
  results.sort_values(inplace = True)
  print(results)
  results.plot(kind = 'barh')
    
combine_feature_importances(rf_model, x_train.columns, cat_var)

### Create predictions using the random forest model

In [ ]:
predict_df = clean_data(test_features)
predict_df = dummy_encode(predict_df, cat_var,num_var)

predictions = rf_model.predict(predict_df)

def save_model_predictions(model,predictions):
    with open('model_predictions.txt', 'w') as file:
        file.write(str(model))
    np.savetxt('..results/predictions.csv', predictions, delimiter = ',')
    
save_model_predictions(rf_model, predictions)